<a href="https://colab.research.google.com/github/jarmi95/03MAIR-Algoritmos-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Jarmi_Recio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Jarmi Recio Martínez  <br>
Url: https://github.com/jarmi95/03MAIR-Algoritmos-Optimizacion/blob/main/Trabajo_Práctico_Algoritmos_Jarmi_Recio.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1p6xZElvV_0kJtUB4ltDEL4Ubhbtqtp_f?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>
>2. <b>Organizar los horarios de partidos de La Liga</b><br>
>3. Configuración de Tribunales

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

Los horarios disponibles se conocen a priori y son los siguientes:

- Viernes: 20
- Sábado: 12, 16, 18, 20
- Domingo: 12, 16, 18, 20
- Lunes: 20

En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores (que tiene relación directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C.

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

|              | Categoría A  | Categoría B  | Categoría C |
|--------------|--------------|--------------|--------------|
| Categoría A  | 2 Millones   | 1,3 Millones | 1 Millón     |
| Categoría B  |              | 0,9 Millones | 0,75 Millones|
| Categoría C  |              |              | 0,47 Millones|


Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla.

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes.

|          | Viernes | Sábado  | Domingo | Lunes   |
|----------|---------|---------|---------|---------|
| 12h      |    -    |   0,55  |   0,45  |    -    |
| 16h      |    -    |   0,7   |   0,75  |    -    |
| 18h      |    -    |   0,8   |   0,85  |    -    |
| 20h      |   0,4   |    1    |    1    |   0,4   |


Es posible la coincidencia de horarios pero en este
caso la audiencia de cada partido se verá afectada y
se estima que se reduce en porcentaje según la
siguiente tabla dependiendo del número de
coincidencias:

| Coincidencias | -% |
|---------------|----|
| 0             | 0%    |
| 1             | 25%   |
| 2             | 45%   |
| 3             | 60%   |
| 4             | 70%   |
| 5             | 75%   |
| 6             | 78%   |
| 7             | 80%   |
| 8             | 80%   |



                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

1. ¿Como represento el espacio de soluciones?

Se entiende por espacio de soluciones el conjunto de todas las posibles soluciones válidas para un problema dado.

En este caso, para todas las posibles combinaciones de enfrentamientos entre equipos de La Liga de Fútbol, que tengan lugar en el horario indicado y siguiendo las directrices de audiencia mencionadas en el enunciado.

Dentro del espacio de soluciones se encuentran las dos principales variables de decisión: el equipo con su correspondiente categoría y el horario de enfrentamiento.

- Equipos: A1, A2, A3, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, C1, C2, C3, C4, C5 y C6.
- Horarios: V20, S12, S16, S18, S20, D12, D16, D18, D20 y L20.

Las múltiples combinaciones que se formar al cruzar dos equipos de la lista en los diferentes horarios propuestos supone el total de soluciones.

Por ejemplo:

SOLUCIÓN 1: [A1 vs A2 - S20, A3 vs B1 - D20, ... ]

SOLUCIÓN 2: [A1 vs A3 - S20, A2 vs B1 - D20, ... ]


In [3]:
# A continuación se definen las variables del espacio de soluciones

Equipos = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']
CatA = ['A1', 'A2', 'A3']
CatB = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']
CatC = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
Horarios = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']


La siguiente función muestra la generaciión aleatoria de partidos, generado un ejemplo de solución en cada ejecución.

In [4]:
import random

def generar_enfrentamientos_aleatorios(equipos, horarios):
  enfrentamientos = []

  equipos_disponibles = equipos.copy()

  while len(equipos_disponibles) >= 2:
    # Seleccionar un horario aleatorio
    horario = random.choice(horarios)

    # Seleccionar dos equipos aleatorios
    equipo1 = random.choice(equipos_disponibles)
    equipo2 = random.choice(equipos_disponibles)

    # Verificar si los equipos ya fueron seleccionados previamente
    while equipo2 == equipo1:
      quipo2 = random.choice(equipos_disponibles)

      # Almacenar el enfrentamiento en la lista
      enfrentamientos.append([horario, equipo1, equipo2])

      # Eliminar los equipos seleccionados de la lista de equipos disponibles
      equipos_disponibles.remove(equipo1)
      equipos_disponibles.remove(equipo2)

  return enfrentamientos

solucion_enfrentamientos = generar_enfrentamientos_aleatorios(Equipos, Horarios)

# Imprimir la solución
for i in solucion_enfrentamientos:
  print(i)

['D18', 'B3', 'B5']
['D20', 'A1', 'B9']
['V20', 'C4', 'B7']
['S20', 'B1', 'C5']
['S20', 'C6', 'B4']
['S12', 'B11', 'C3']
['V20', 'B10', 'A2']
['S20', 'B6', 'C2']
['S18', 'B2', 'C1']
['S16', 'A3', 'B8']


2. ¿Cual es la función objetivo?

Una vez conocido el espacio de posibles soluciones, es necesario ajustarlo al objetivo del problema, es decir, maximizar la audiencia de la jornada de La Liga.

Para conseguir la combinación de partidos entre equipos y horarios que obtenga la mayor audiencia posible en la jornada es necesario tener en cuenta tres factores más:
- Estadísticas asociadas a categoría (Est_Cat): las audiencias son mayores para enfrentamientos entre equipos de mayor categoría.
- Estadísticas asociadas al horario (Est_Hor): las audiendias son máximas, en coeficiente, sábado y domingo a las 20h.
- Estadísticas de coincidencia (Est_Coin): las coincidencias entre dos o más partidos en una franja horaria supone una reducción de la audiencia.

Por tanto, la función a maximizar puede definirse como:

F = SUM(Est_Cat * Est_Hor * Est_Coin)

In [5]:
def audiencia_base(local, visitante, slot):

  # Se ifentifican los equipos y su audiencia por petenencia a categoría
  if local in CatA and visitante in CatA:
    aud_base = 2
  elif local in CatB and visitante in CatB:
    aud_base = 0.9
  elif local in CatC and visitante in CatC:
    aud_base = 0.47
  elif (local in CatA and visitante in CatB) or (local in CatB and visitante in CatA):
    aud_base = 1.3
  elif (local in CatA and visitante in CatC) or (local in CatC and visitante in CatA):
    aud_base = 1
  elif (local in CatB and visitante in CatC) or (local in CatC and visitante in CatB):
    aud_base = 0.75

  # Se aplica la corrección por horario
  horarios = {'S20': 1, 'D20': 1, 'V20': 0.4, 'L20': 0.4, 'S12': 0.55, 'S16': 0.7, 'S18': 0.8, 'D12': 0.45, 'D16': 0.75, 'D18': 0.85}
  aud_base_slot = aud_base * horarios.get(slot, 0)

  return aud_base_slot

# Ejemplo de cálculo: partido de máxima audiencia
aud_base_slot = audiencia_base('A1', 'A2', 'S20')
print(aud_base_slot)

2


A continuación se calculan las audiencias de cada enfrantamiento antes de aplicar restricciones:

In [6]:
aud_total = 0

# Se genera una jornada aleatoria
solucion_enfrentamientos = generar_enfrentamientos_aleatorios(Equipos, Horarios)

# Se muestran las audiencias por partido sin corregir
for i in solucion_enfrentamientos:
  aud_base = round(audiencia_base(i[1],i[2], i[0]), 4)
  aud_total += aud_base
  print(f'Equipo: {i[1]} vs equipo: {i[2]} en horario: {i[0]} -> {aud_base}')

print(f'Audiencia total de la jornada: {aud_total}')

Equipo: B2 vs equipo: C4 en horario: L20 -> 0.3
Equipo: C3 vs equipo: B3 en horario: S16 -> 0.525
Equipo: B1 vs equipo: B10 en horario: D16 -> 0.675
Equipo: C1 vs equipo: B6 en horario: L20 -> 0.3
Equipo: C6 vs equipo: B11 en horario: S18 -> 0.6
Equipo: B9 vs equipo: B4 en horario: D20 -> 0.9
Equipo: C5 vs equipo: B8 en horario: L20 -> 0.3
Equipo: A2 vs equipo: A3 en horario: D12 -> 0.9
Equipo: B7 vs equipo: A1 en horario: S12 -> 0.715
Equipo: B5 vs equipo: C2 en horario: L20 -> 0.3
Audiencia total de la jornada: 5.515


3. ¿Como implemento las restricciones?

Las principales restricciones a incluir, además de las audiencias base y los coeficientes comentados anteriormente, están relacionados con:

3.1 La no repetición de ningún equipo dentro de una misma jornada -> Ya implementada anteriormente dada la selección única por equipo y jornada.

3.2 La obligatoriedad de un partido en horario de viernes y lunes pese a tener el coeficiente más bajo -> Se crea una función que corrige la no existencia de enfrentamientos los viernes y lunes a las 20:00.


In [7]:
import random

def check_VL20(enfrentamientos):
  # Verificar si 'V20' está presente en algún enfrentamiento
  v20_presente = any('V20' in sublista for sublista in enfrentamientos)

  # Verificar si 'L20' está presente en algún enfrentamiento
  l20_presente = any('L20' in sublista for sublista in enfrentamientos)

  # Si 'V20' no está presente, incluirlo en alguna sublista
  if not v20_presente:
    indice = random.randint(0, len(enfrentamientos) - 1)
    if enfrentamientos[indice][0] != 'L20':
      enfrentamientos[indice][0] = 'V20'
    else:
      enfrentamientos[indice - 1][0] = 'V20'

    # Si 'L20' no está presente, incluirlo en alguna sublista
    if not l20_presente:
      indice = random.randint(0, len(enfrentamientos) - 1)
      if enfrentamientos[indice][0] != 'V20':
        enfrentamientos[indice][0] = 'L20'
      else:
        enfrentamientos[indice - 1][0] = 'L20'

    return enfrentamientos

In [8]:

solucion_enfrentamientos = generar_enfrentamientos_aleatorios(Equipos, Horarios)
print('\nSin incluir corrección horaria:')
# Imprimir la solución sin corrección de horarios
for i in solucion_enfrentamientos:
  print(i)

solucion_enfrentamientos = check_VL20(solucion_enfrentamientos)
print('\nIncluyendo corrección horaria:')
# Imprimir la solución con corrección de horarios
for i in solucion_enfrentamientos:
  print(i)


Sin incluir corrección horaria:
['V20', 'B1', 'B4']
['V20', 'A1', 'A2']
['D16', 'C5', 'B3']
['D20', 'B10', 'B7']
['L20', 'B6', 'C4']
['D16', 'B5', 'C3']
['S16', 'C1', 'B8']
['V20', 'C6', 'B2']
['S20', 'B9', 'A3']
['S20', 'B11', 'C2']

Incluyendo corrección horaria:
['V20', 'B1', 'B4']
['V20', 'A1', 'A2']
['D16', 'C5', 'B3']
['D20', 'B10', 'B7']
['L20', 'B6', 'C4']
['D16', 'B5', 'C3']
['S16', 'C1', 'B8']
['V20', 'C6', 'B2']
['S20', 'B9', 'A3']
['S20', 'B11', 'C2']


3.3 La correcta aplicación de reducciones de audiencia en base a las coincidencias dadas.

La siguiente celda muestra el generador actual de jornadas y audiencias, donde no se cuenta con las reducciones de audiencia por solapamiento horario

In [9]:
solucion_enfrentamientos = generar_enfrentamientos_aleatorios(Equipos, Horarios)
solucion_enfrentamientos = check_VL20(solucion_enfrentamientos)
#audiencias_jornada = solucion_enfrentamientos.copy()

for ind, i in enumerate(solucion_enfrentamientos):
  aud_base = round(audiencia_base(i[1],i[2], i[0]), 4)
  #audiencias_jornada[ind].append(aud_base)
  aud_total += aud_base
  print(f'Equipo: {i[1]} vs equipo: {i[2]} en horario: {i[0]} -> {aud_base}')

print(f'Audiencia total de la jornada: {aud_total}')

Equipo: B2 vs equipo: C5 en horario: S16 -> 0.525
Equipo: A2 vs equipo: C3 en horario: L20 -> 0.4
Equipo: B8 vs equipo: B10 en horario: S12 -> 0.495
Equipo: B4 vs equipo: C4 en horario: D16 -> 0.5625
Equipo: C1 vs equipo: B6 en horario: S16 -> 0.525
Equipo: C6 vs equipo: B3 en horario: S20 -> 0.75
Equipo: A3 vs equipo: A1 en horario: V20 -> 0.8
Equipo: B1 vs equipo: C2 en horario: D18 -> 0.6375
Equipo: B11 vs equipo: B5 en horario: S12 -> 0.495
Equipo: B7 vs equipo: B9 en horario: S20 -> 0.9
Audiencia total de la jornada: 11.605


A continuación se aplica dicha reducción en base a los factores vistos en la descripción del problema:

In [30]:
def aud_finales(audiencias_jornada):

  # Se crea un contador para contabilizar las repeticiones en horario
  contador = {}
  for sublista in audiencias_jornada:
    primera_posicion = sublista[0]
    if primera_posicion in contador:
      contador[primera_posicion] += 1
    else:
     contador[primera_posicion] = 1

    # Se aplica el factor de solapamiento para cada fecha con más de un enfrentamiento
    for clave, valor in contador.items():
      if valor > 1:
        indices_coincidentes = [i for i, sublista in enumerate(audiencias_jornada) if sublista[0] == clave]
        # Multiplicar por el factor
        for indice in indices_coincidentes:
          if valor == 2:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.75,4)
          elif valor == 3:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.55,4)
          elif valor == 4:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.4,4)
          elif valor == 5:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.3,4)
          elif valor == 6:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.25,4)
          elif valor == 7:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.22,4)
          elif valor == 8:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.2,4)
          elif valor == 9:
            audiencias_jornada[indice][-1] = round(audiencias_jornada[indice][-1] * 0.2,4)

  return audiencias_jornada

In [38]:
# Funciones comentadas anteriormente
solucion_enfrentamientos = generar_enfrentamientos_aleatorios(Equipos, Horarios)
solucion_enfrentamientos = check_VL20(solucion_enfrentamientos)
audiencias_jornada = solucion_enfrentamientos.copy()

total = 0
total_corregido = 0

print('\nAudiencias sin corrección de solapamiento:')
for ind, i in enumerate(solucion_enfrentamientos):
  aud_base = round(audiencia_base(i[1],i[2], i[0]), 4)
  total += aud_base
  audiencias_jornada[ind].append(aud_base)
  print(audiencias_jornada[ind])
print(f'La audiencia total es: {round(total,4)}')

# Se aplica la función de solapamiento
aud_fin_corregidas = aud_finales(audiencias_jornada)
print('\nAudiencias con corrección de solapamiento:')
for i in aud_fin_corregidas:
  total_corregido += i[-1]
  print(i)
print(f'La audiencia total corregida es: {round(total_corregido,4)}')


Audiencias sin corrección de solapamiento:
['V20', 'B1', 'A2', 0.52]
['S18', 'B9', 'B6', 0.72]
['S16', 'B3', 'A3', 0.91]
['S16', 'C1', 'B4', 0.525]
['D18', 'B11', 'B8', 0.765]
['D18', 'C2', 'B5', 0.6375]
['S20', 'C3', 'B10', 0.75]
['L20', 'C4', 'C6', 0.188]
['S16', 'B7', 'C5', 0.525]
['S20', 'B2', 'A1', 1.3]
La audiencia total es: 6.8405

Audiencias con corrección de solapamiento:
['V20', 'B1', 'A2', 0.52]
['S18', 'B9', 'B6', 0.72]
['S16', 'B3', 'A3', 0.0653]
['S16', 'C1', 'B4', 0.0377]
['D18', 'B11', 'B8', 0.1815]
['D18', 'C2', 'B5', 0.1513]
['S20', 'C3', 'B10', 0.5625]
['L20', 'C4', 'C6', 0.188]
['S16', 'B7', 'C5', 0.0377]
['S20', 'B2', 'A1', 0.975]
La audiencia total corregida es: 3.439


## Código resumido

In [45]:
def jornada_random_corregida(Equipos, Horarios):

  # Generación aleatoria de encuentros
  duelos_aleatorios = generar_enfrentamientos_aleatorios(Equipos, Horarios)

  # Forzar la aparición de encuentros Lunes y Viernes
  duelos_LV = check_VL20(duelos_aleatorios)

  # Calcular audiencias base
  for ind, i in enumerate(duelos_LV):
    aud_base = round(audiencia_base(i[1],i[2], i[0]), 4)
    duelos_LV[ind].append(aud_base)

  # Calcular las audiencias tras solapamiento de partidos
  audiencias_finales_corregidas = aud_finales(duelos_LV)

  return audiencias_finales_corregidas

audiencia_jornada = 0

jornada_random = jornada_random_corregida(Equipos, Horarios)

print('\nAudiencias finales corregidas:')
for i in jornada_random:
  audiencia_jornada += i[-1]
  print(i)
print(f'La audiencia total corregida es: {round(audiencia_jornada,4)}')


Audiencias finales corregidas:
['S16', 'B3', 'B4', 0.0841]
['D16', 'C2', 'A1', 0.4219]
['S18', 'B10', 'C4', 0.6]
['S16', 'B7', 'B9', 0.0841]
['D12', 'A2', 'B8', 0.4387]
['V20', 'B1', 'C3', 0.3]
['D20', 'B6', 'B5', 0.9]
['L20', 'C1', 'C5', 0.188]
['D16', 'B11', 'C6', 0.3164]
['D12', 'A3', 'B2', 0.4387]
La audiencia total corregida es: 3.7719


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [12]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [13]:
#Respuesta